In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'ieee.dss'
#fn = '02node_threephase_unbalanced_edit.dss'

times = np.linspace(0, 2*np.pi, 5)
der = [0, 0, 0, 0, 0]
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode))
INR01=  np.zeros((len(times), 3, nline))
STXNR01 = np.zeros((len(times), 3, nline))
SRXNR01 = np.zeros((len(times), 3, nline))
iNR01 = np.zeros((len(times), 3, nnode))
sNR01 = np.zeros((len(times), 3, nnode))


X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None) #generate the all the matrices

for i in range(len(times)):
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i]) 
    #run NR3 with variations in time, DER, and capacitance
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91821246-0.10957476j  0.94941713-0.05560837j
   0.        +0.j          0.        +0.j          0.90898503-0.11336917j
   0.90169707-0.11722609j  0.        +0.j          0.91005304-0.10758234j
   0.94379241-0.07218327j  0.99789667-0.00254538j  0.95583011-0.05345307j
   0.91821246-0.10957476j  0.95262362-0.05453072j  0.91603083-0.10972829j]
 [-0.5       -0.8660254j  -0.53449743-0.85186265j -0.51752657-0.83897394j
  -0.51336593-0.83343754j -0.51205107-0.83131772j -0.53898377-0.85237005j
  -0.54286784-0.85266959j  0.        +0.j          0.        +0.j
  -0.52311502-0.84534436j -0.50086179-0.8649375j  -0.51809758-0.84317945j
  -0.53449743-0.85186265j -0.51781207-0.84107669j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.38105732+0.80661364j -0.42607817+0.830691j
  -0.42587663+0.83523106j -0.4241427 +0.83395432j -0.3800959

C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part



Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91702735-0.11069987j  0.94871795-0.05618052j
   0.        +0.j          0.        +0.j          0.90767991-0.11450445j
   0.90029322-0.11837454j  0.        +0.j          0.90878072-0.10867913j
   0.94298277-0.07292763j  0.99786666-0.00257168j  0.95519986-0.05400536j
   0.91702735-0.11069987j  0.95195891-0.05509294j  0.91482195-0.11085709j]
 [-0.5       -0.8660254j  -0.53477608-0.85172867j -0.51766227-0.83871129j
  -0.51345692-0.83311909j -0.51212784-0.83097858j -0.53929195-0.85223708j
  -0.54319887-0.85253567j  0.        +0.j          0.        +0.j
  -0.52329715-0.8451453j  -0.50086855-0.86492693j -0.51823945-0.84295744j
  -0.53477608-0.85172867j -0.51795086-0.84083436j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.37968538+0.80546426j -0.42523648+0.83003544j
  -0.42503157+0.83462318j -0.42327961+0.83333112j -0.37872079+0.80208613j
  -0.3778

  -1.01364759e-01+0.16906905j  5.38666666e-29+0.j        ]]
STXNR:
[[ 1.32984891e+00+0.70618545j  1.32880668e+00+0.70120319j
   1.14402869e+00+0.48820044j  1.12329679e+00+0.44856448j
  -4.75272702e-29+0.j          1.63933488e-01+0.1133575j
   1.63535646e-01+0.11275737j -8.94449221e-30+0.j
   0.00000000e+00+0.j          5.90282228e-01+0.07719988j
   4.98311954e-01-0.00368469j  1.31825639e-01+0.08835616j
  -4.34962866e-29+0.j          1.31553454e-01+0.08808474j]
 [ 9.35492387e-01+0.44886422j  9.35592518e-01+0.44740183j
   5.20969954e-01+0.12836575j  4.56486614e-01+0.08853503j
   2.71786820e-29+0.j          1.22680699e-01+0.09235117j
   1.22517026e-01+0.09205809j  2.93944481e-01+0.19743712j
   1.17561640e-01+0.06746871j  6.98960823e-02-0.13764388j
   6.96044753e-02-0.13828248j  3.92673897e-29+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 1.41086362e+00+0.69328289j  1.40813599e+00+0.68800466j
   1.11303870e+00+0.42208162j  9.77878316e-01+0.33107j
   3.72099696e-29+0.j  

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91355314-0.1139753j   0.94666998-0.05784661j
   0.        +0.j          0.        +0.j          0.90385492-0.11780794j
   0.89617956-0.12171506j  0.        +0.j          0.9050524 -0.11187042j
   0.94060956-0.07509525j  0.99777871-0.00264828j  0.95335281-0.0556139j
   0.91355314-0.1139753j   0.95001139-0.05673025j  0.9112784 -0.11414289j]
 [-0.5       -0.8660254j  -0.53559099-0.85135395j -0.51805862-0.83795666j
  -0.51372302-0.83220229j -0.51235252-0.8300018j  -0.54019321-0.85186517j
  -0.54416694-0.85216087j  0.        +0.j          0.        +0.j
  -0.52382936-0.84457743j -0.50088827-0.8648966j  -0.51865374-0.84232058j
  -0.53559099-0.85135395j -0.51835618-0.84013862j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.37567776+0.80208615j -0.42277861+0.82810818j
  -0.42256379+0.83283483j -0.42075931+0.83149785j -0.374703

In [3]:
#fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'ieee.dss'
#fn = '02node_threephase_unbalanced_edit.dss'

VNR02 = np.zeros((len(times), 3, nnode))
INR02=  np.zeros((len(times), 3, nline))
STXNR02 = np.zeros((len(times), 3, nline))
SRXNR02 = np.zeros((len(times), 3, nline))
iNR02 = np.zeros((len(times), 3, nnode))
sNR02 = np.zeros((len(times), 3, nnode))

for i in range(len(times)):
    network1 = nmf2(fn, times[i])
    #network1 = network_mapper_function('compare__lindist3flow_05node_threephase_unbalanced_oscillation_03.dss')

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =1*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

BASE
Vbase: 2401.7771198288433
Sbase: 1000000.0
Ibase: 416.358367204057
Zbase: 5.768533333333334

NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc

12 - FZpu:
 [[0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.01309222+0.01327247j]]
13 - FZpu:
 [[0.03526193+0.01345863j 0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]]
ADMITTANCE
0 - FYpu:
 [[132.11316623-380.79156025j -56.05356967+140.78546783j
  -30.69531751+105.24280468j]
 [-56.05356967+140.78546783j 115.6831699 -360.83761211j
  -14.57183636 +80.37857593j]
 [-30.69531751+105.24280468j -14.57183636 +80.37857593j
  102.30104282-340.41112286j]]
1 - FYpu:
 [[ 6.60565831-19.03957801j -2.80267848 +7.03927339j
  -1.53476588 +5.26214023j]
 [-2.80267848 +7.03927339j  5.78415849-18.04188061j
  -0.72859182 +4.0189288j ]
 [-1.53476588 +5.26214023j -0.72859182 +4.0189288j
   5.11505214-17.02055614j]]
2 - FYpu:


C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: ComplexWarning: Casting complex values to real discards the imaginary part


NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'ac', 'c', 'a']
PH:
 [[1 1 1 1 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1


LOADS
aPQ:
 [[0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]]
aI:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
aZ:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
ppu:
 [[0.    0.385 0.16  0.    0.    0.085 0.485 0.    0.128 0.017 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.17  0.115 0.    0.068 0.    0.    0.066 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.    0.115 0.085 0.29  0.17  0.    0.117 0.    0.
  0.    0.    0.   ]]
qpu:
 [[0.     0.22   0.11   0.     0.     0.0755 0.19   0.     0.086  0.01
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.125  0.066  0.     0.06   0.     0.     0.038
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.     0.066 

 [[0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.01309222+0.01327247j]]
13 - FZpu:
 [[0.03526193+0.01345863j 0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]]
ADMITTANCE
0 - FYpu:
 [[132.11316623-380.79156025j -56.05356967+140.78546783j
  -30.69531751+105.24280468j]
 [-56.05356967+140.78546783j 115.6831699 -360.83761211j
  -14.57183636 +80.37857593j]
 [-30.69531751+105.24280468j -14.57183636 +80.37857593j
  102.30104282-340.41112286j]]
1 - FYpu:
 [[ 6.60565831-19.03957801j -2.80267848 +7.03927339j
  -1.53476588 +5.26214023j]
 [-2.80267848 +7.03927339j  5.78415849-18.04188061j
  -0.72859182 +4.0189288j ]
 [-1.53476588 +5.26214023j -0.72859182 +4.0189288j
   5.11505214-17.02055614j]]
2 - FYpu:
 [[19.80707

 [0.00256092+0.00823599j 0.00251988+0.00631858j 0.01220706+0.01988326j]]
7 - FZpu:
 [[0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.02173172+0.0222751j  0.00339158+0.00753666j]
 [0.        +0.j         0.00339158+0.00753666j 0.02182365+0.02211422j]]
8 - FZpu:
 [[0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.01303903+0.01336506j 0.00203495+0.004522j  ]
 [0.        +0.j         0.00203495+0.004522j   0.01309419+0.01326853j]]
9 - FZpu:
 [[0.01299702+0.00719606j 0.00522816+0.00045446j 0.00465316-0.00030239j]
 [0.00522816+0.00045446j 0.01283168+0.00651224j 0.00522816+0.00045446j]
 [0.00465316-0.00030239j 0.00522816+0.00045446j 0.01299702+0.00719606j]]
10 - FZpu:
 [[0.01299702+0.00719606j 0.00522816+0.00045446j 0.00465316-0.00030239j]
 [0.00522816+0.00045446j 0.01283168+0.00651224j 0.00522816+0.00045446j]
 [0.00465316-0.00030239j 0.00522816+0.00045446j 0.01299702+0.00719606j]]
11 - FZpu:
 [[


LOADS
aPQ:
 [[0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]]
aI:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
aZ:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
ppu:
 [[0.    0.385 0.16  0.    0.    0.085 0.485 0.    0.128 0.017 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.17  0.115 0.    0.068 0.    0.    0.066 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.    0.115 0.085 0.29  0.17  0.    0.117 0.    0.
  0.    0.    0.   ]]
qpu:
 [[0.     0.22   0.11   0.     0.     0.0755 0.19   0.     0.086  0.01
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.125  0.066  0.     0.06   0.     0.     0.038
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.     0.066 

In [4]:
VNR01.shape

(5, 3, 15)

In [5]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
for n in range(len(dss.Lines.AllNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[n])
    print(dss.Lines.AllNames()[n])

sbs_650
650_632
632_670
670_671
671_680
632_633
633_634
632_645
645_646
671_692
692_675
671_684
684_611
684_652


In [6]:
VDSS0 = np.zeros((len(times), 3, nnode))
IDSS0 =  np.zeros((len(times), 3, nline))
SRXDSS0 = np.zeros((len(times), 3, nline))
STXDSS0 = np.zeros((len(times), 3, nline))

kW_list = np.array([])
kvar_list = np.array([])
for k in range(len(dss.Loads.AllNames())):
    dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    dss.Loads.Name(dss.Loads.AllNames()[k])
    kW_list = np.append(kW_list, dss.Loads.kW())
    kvar_list = np.append(kvar_list, dss.Loads.kvar())



for i in range(len(times)):
    #dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss') #Using OpenDSS
    dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    #dss.run_command('Redirect ieee.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_edit.dss')
    
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
    Vbase = dss.Bus.kVBase() * 1000
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase
    
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    kWcurr = dss.Loads.kW()
    kvarcurr = dss.Loads.kvar()
    
    #Time varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        
        dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])) )
        dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
    dss.Solution.Convergence(0.000000000001)
    
    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        #set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        #Easy process to get all names and count of loads, a trick to avoid
        #some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')
    
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
        

        Vtemp = np.asarray(dss.Bus.PuVoltage())

        Vtemp = Vtemp[0:5:2] + 1j*Vtemp[1:6:2]
        

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:5:2] + 1j*Imn[1:6:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:5:2] + 1j*STXtemp[1:6:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:5:2] + 1j*SRXtemp[1:6:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  9
Tolerance:  1e-12

15
['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
15
['671', '634a', '634b', '634c', '645', '646', '692', '675a', '675b', '675c', '611', '652', '670a', '670b', '670c']

VDSS
 [[ 0.999808-2.31000e-04j  0.878117-1.33969e-01j  0.931033-6.53090e-02j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.868665-1.39676e-01j
   0.863452-1.45725e-01j  0.      +0.00000e+00j  0.870902-1.32021e-01j
   0.918081-8.68860e-02j  0.996836-3.23700e-03j  0.93739 -6.33560e-02j
   0.878117-1.33969e-01j  0.934211-6.43320e-02j  0.876138-1.34097e-01j]
 [-0.50012 -8.65773e-01j -0.581687-7.94743e-01j -0.539743-8.02596e-01j
  -0.531896-7.91948e-01j -0.528455-7.86690e-01j -0.59088 -7.94882e-01j
  -0.598735-7.94674e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
  -0.553811-8.02101e-01j -0.502044-8.62972e-01j -0.540518-8.06948e-01j
  -0

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  10
Tolerance:  1e-12

15
['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
15
['671', '634a', '634b', '634c', '645', '646', '692', '675a', '675b', '675c', '611', '652', '670a', '670b', '670c']

VDSS
 [[ 0.999803-2.34000e-04j  0.875156-1.35196e-01j  0.929405-6.58410e-02j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.865498-1.40873e-01j
   0.860147-1.46908e-01j  0.      +0.00000e+00j  0.867829-1.33211e-01j
   0.916092-8.76330e-02j  0.996759-3.26400e-03j  0.935875-6.38650e-02j
   0.875156-1.35196e-01j  0.93264 -6.48530e-02j  0.873145-1.35330e-01j]
 [-0.500121-8.65767e-01j -0.582297-7.92251e-01j -0.539979-8.00766e-01j
  -0.531976-7.89939e-01j -0.528467-7.84592e-01j -0.591525-7.92365e-01j
  -0.599392-7.92130e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
  -0.554181-8.00098e-01j -0.502057-8.62884e-01j -0.54078 -8.05216e-01j
  -

C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:163: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:165: ComplexWarning: Casting complex values to real discards the imaginary part


In [7]:
# dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
# cap_dict = {}
# cap_ph_dict = {}
# for n in range(len(dss.Capacitors.AllNames())):
#     dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
#     #print(dss.CktElement.BusNames()[0])
#     pattern =  r"(\w+)\."  
#     m = re.findall(pattern, dss.CktElement.BusNames()[0])
#     cap_dict[m[0]] = dss.Capacitors.kvar()    
#     load_phases = [0, 0, 0]
#     for i in range(1, 4): #if the phase is present, what other phases are
#         pattern = r"\.%s" % (str(i))
#         m2 = re.findall(pattern, dss.CktElement.BusNames()[0])
#         if m2:
#             load_phases[i - 1] = 1
#     cap_ph_dict[m[0]] = load_phases
    

for k2 in range(1, len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k2])
    #print(len(dss.Lines.AllNames()))
    lc = dss.Lines.LineCode()
    dss.LineCodes.Name(lc)
    
    print(lc)
    print(dss.LineCodes.Xmatrix())
        
        


mtx601
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
mtx601
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
mtx601
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
mtx601
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
mtx602
[1.1814, 0.4236, 0.5017, 0.4236, 1.1983, 0.3849, 0.5017, 0.3849, 1.2112]
mtx602
[1.1814, 0.4236, 0.5017, 0.4236, 1.1983, 0.3849, 0.5017, 0.3849, 1.2112]
mtx603
[1.3569, 0.4591, 0.4591, 1.3471]
mtx603
[1.3569, 0.4591, 0.4591, 1.3471]
mtx606
[0.438352, 0.0276838, -0.0184204, 0.0276838, 0.396697, 0.0276838, -0.0184204, 0.0276838, 0.438352]
mtx606
[0.438352, 0.0276838, -0.0184204, 0.0276838, 0.396697, 0.0276838, -0.0184204, 0.0276838, 0.438352]
mtx604
[1.3569, 0.4591, 0.4591, 1.3471]
mtx605
[1.3475]
mtx607
[0.5124]


In [8]:
# INR01[0].imag-IDSS[0].imag


In [9]:
#dss - opendss solution
#01 - vectorized solution without network mapper
#02 - nonvectorized solution using OpenDSS and network mapper
for i in range(len(times)):
    print(i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

0
Complex Voltage difference
1.1102230246251565e-16
0.05586681963504447
0.055866819635044584
Complex Current difference
2.220446049250313e-16
0.5931404552489737
0.5931404552489737
Complex TX Power difference
2.220446049250313e-16
0.8123880305659268
0.8123880305659266
Complex RX Power difference
2.220446049250313e-16
0.8117598412668653
0.8117598412668652
Voltage Magnitude difference
1.1102230246251565e-16
0.05586681963504447
0.055866819635044584
Voltage Angle difference
0.0
0.0
0.0


1
Complex Voltage difference
2.220446049250313e-16
0.055868221450311006
0.055868221450310895
Complex Current difference
2.220446049250313e-16
0.6024576544439233
0.6024576544439233
Complex TX Power difference
2.220446049250313e-16
0.8185145167106773
0.8185145167106773
Complex RX Power difference
2.220446049250313e-16
0.8178636153765547
0.8178636153765547
Voltage Magnitude difference
2.220446049250313e-16
0.055868221450311006
0.055868221450310895
Voltage Angle difference
0.0
0.0
0.0


2
Complex Voltage differ

In [10]:
#with capacitors and fixed Vbase
0
Complex Voltage difference
1.1102230246251565e-16
0.05586681963504425
0.05586681963504436
Complex Current difference
2.220446049250313e-16
0.5931404552489737
0.5931404552489737
Complex TX Power difference
2.220446049250313e-16
0.8123880305659268
0.8123880305659266
Complex RX Power difference
2.220446049250313e-16
0.8117598412668653
0.8117598412668652
Voltage Magnitude difference
1.1102230246251565e-16
0.05586681963504425
0.05586681963504436
Voltage Angle difference
0.0
0.0
0.0


1
Complex Voltage difference
2.220446049250313e-16
0.05538227509724125
0.05538227509724114
Complex Current difference
2.220446049250313e-16
0.6107983915176805
0.6107983915176805
Complex TX Power difference
2.220446049250313e-16
0.816962303971847
0.816962303971847
Complex RX Power difference
2.220446049250313e-16
0.8163047774389821
0.8163047774389821
Voltage Magnitude difference
2.220446049250313e-16
0.05538227509724125
0.05538227509724114
Voltage Angle difference
0.0
0.0
0.0


2
Complex Voltage difference
1.6653345369377348e-16
0.05488425108938877
0.05488425108938877
Complex Current difference
2.498001805406602e-16
0.6284001737075392
0.6284001737075391
Complex TX Power difference
3.3306690738754696e-16
0.821407623944402
0.8214076239444019
Complex RX Power difference
2.220446049250313e-16
0.8207200949623612
0.8207200949623612
Voltage Magnitude difference
1.6653345369377348e-16
0.05488425108938877
0.05488425108938877
Voltage Angle difference
0.0
0.0
0.0


3
Complex Voltage difference
1.1102230246251565e-16
0.054378392188340396
0.054378392188340285
Complex Current difference
2.220446049250313e-16
0.6457684270995072
0.6457684270995073
Complex TX Power difference
2.220446049250313e-16
0.8256843931422386
0.8256843931422384
Complex RX Power difference
4.440892098500626e-16
0.8249665133346354
0.8249665133346353
Voltage Magnitude difference
1.1102230246251565e-16
0.054378392188340396
0.054378392188340285
Voltage Angle difference
0.0
0.0
0.0


4
Complex Voltage difference
1.1102230246251565e-16
0.05388060298691688
0.05388060298691699
Complex Current difference
2.220446049250313e-16
0.6621385961092344
0.6621385961092344
Complex TX Power difference
2.220446049250313e-16
0.8293481661551416
0.8293481661551416
Complex RX Power difference
2.220446049250313e-16
0.8286020197061426
0.8286020197061426
Voltage Magnitude difference
1.1102230246251565e-16
0.05388060298691688
0.05388060298691699
Voltage Angle difference
0.0
0.0
0.0

SyntaxError: invalid syntax (<ipython-input-10-736b29cab948>, line 3)

In [ ]:
VNR01[0]

In [ ]:
VNR02[0]

In [ ]:
VDSS0[0]

In [ ]:
x = np.array([[1, 2], [4, 4]])
y = np.array([[3, 3], [3, 5]])
print(np.multiply(x, y))